In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import pickle
import imageio

from tqdm import trange
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path

from data_prep import LabeledVideoDataset
from text_processing import doTextPart, getGloveEmbeddings, selectTemplates

from trainer import to_video
from models import *

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
path = '/home/lukoshkin/Datasets/clean-20bn-sth-sth/metadata/validation.json'
cache = '/home/lukoshkin/TVGAN/logdir/'

templates = ['Pushing [something] from left to right']
new_name = selectTemplates(path, templates, '1-template.pkl')

v2tds = LabeledVideoDataset(new_name, cache, video_shape=(32, 32, 32, 3))

Preparing dataset: 100%|██████████| 93/93 [00:03<00:00, 24.98it/s]


No of corrupted videos 8
Caching database to 1-template.db
Done!


In [3]:
device = torch.device("cuda:0")
batch_size = 12

data = v2tds.data
data = np.rec.array(data)
slens = torch.tensor(data.f0, device=device)
label = torch.tensor(data.f1, device=device)
video = torch.tensor(data.f2, device=device)

cache = Path(cache)
with open(cache / 'vocab.pkl', 'rb') as fp:
    t2i = pickle.load(fp)
    max_sen_len = int(fp.readline())

emb_weights = getGloveEmbeddings('../embeddings', cache, t2i)
emb_weights = torch.tensor(emb_weights, device=device)
encoder = SimpleTextEncoder(emb_weights)
embeddings = encoder(label, slens)

class BN20sthsth(Dataset):
    def __init__(self):
        self.embs = embeddings
        self.gifs = video
        
    def __getitem__(self, idx):
        return self.embs[idx], self.gifs[idx]
    
    def __len__(self):
        return len(embeddings)
    
ds = BN20sthsth()
dl = DataLoader(ds, batch_size, shuffle=True)
        
val_samples = [168029, 157604, 71563, 82109]
val_samples = v2tds.getById(val_samples)
lens = torch.tensor(val_samples.f0, device=device)
lbls = torch.tensor(val_samples.f1, device=device)
test = encoder(lbls, lens)

Reading glove embeddings: 100%|██████████| 400001/400001 [00:06<00:00, 58552.20it/s]


In [5]:
gen = SimpleVideoGenerator(dim_Z=50, cond_size=50)
i_dis = StackImageDiscriminator(cond_size=50)
v_dis = StackVideoDiscriminator(cond_size=50)

gen = gen.to(device)
i_dis = i_dis.to(device)
v_dis = v_dis.to(device)

lr_g = 0.00005
lr_d = 0.0002

g_opt = optim.Adam(gen.parameters(), lr=lr_g, betas=(0.3, 0.999))
i_opt = optim.Adam(i_dis.parameters(), lr=lr_d, betas=(0.3, 0.999))
v_opt = optim.Adam(v_dis.parameters(), lr=lr_d, betas=(0.3, 0.999))

In [ ]:
!rm -r runs/process_0

loss = {}
k_d, k_g = 2, 1
num_epochs = 10000
writer = SummaryWriter('runs/process_0')

for epoch in range(num_epochs):
    print(f'EPOCH: {epoch} / {num_epochs}')
    for e, v in dl:
        ne = torch.roll(e, 1, 0)
        for _ in range(k_d):
            with torch.no_grad():
                gv = gen(e)
            # i = image, v = video,  e = embedding, g = generated,
            # s = scores, n = negative, p = positive
            pis, nis, gis = i_dis(v, e), i_dis(v, ne), i_dis(gv, e)
            pvs, nvs, gvs = v_dis(v, e), v_dis(v, ne), v_dis(gv, e)
            
            l1 = (pis.log() + pvs.log()).mean()
            l2 = ((-nis).log1p() + (-nvs).log1p()).mean()
            l3 = ((-gis).log1p() + (-gvs).log1p()).mean()
            L = -.33 * (l1 + l2 + l3)
            
            if np.isnan(L.item()):
                print(L)
                print(pis, nis, gis)
                print(pvs, nvs, gvs)
            
            i_opt.zero_grad()
            v_opt.zero_grad()
            L.backward()
            i_opt.step()
            v_opt.step()
            
        loss['D'] = L.item()
        
        for _ in range(k_g):
            gv = gen(e)
            L = -.5 * (i_dis(gv, e).log() + v_dis(gv, e).log()).mean()
            
            if np.isnan(L.item()):
                print(L)
                print(i_dis(gv, e))
                print(v_dis(gv, e))
            
            g_opt.zero_grad()
            L.backward()
            g_opt.step()
            
        loss['G'] = L.item()
    
    if epoch % 4 == 0:
        gen.eval()
        with torch.no_grad():
            gv = gen(test)
        writer.add_scalars('Loss_ap', loss, epoch)
        writer.add_video('Fakes_ap', to_video(gv), epoch)
        gen.train()

EPOCH: 0 / 10000


MoviePy - Building file /tmp/tmpef4ds3bk.gif with imageio.
EPOCH: 1 / 10000


EPOCH: 2 / 10000
EPOCH: 3 / 10000
EPOCH: 4 / 10000


MoviePy - Building file /tmp/tmp0zuxvzn5.gif with imageio.


EPOCH: 5 / 10000
EPOCH: 6 / 10000
EPOCH: 7 / 10000
EPOCH: 8 / 10000


...

EPOCH: 1618 / 10000
EPOCH: 1619 / 10000
EPOCH: 1620 / 10000


MoviePy - Building file /tmp/tmplh0ur84t.gif with imageio.


EPOCH: 1621 / 10000
EPOCH: 1622 / 10000
EPOCH: 1623 / 10000
EPOCH: 1624 / 10000


MoviePy - Building file /tmp/tmp6ja55mql.gif with imageio.
EPOCH: 1625 / 10000


EPOCH: 1626 / 10000
EPOCH: 1627 / 10000
EPOCH: 1628 / 10000
